# Regressione lineare multivariata (prezzi immobili)

Obiettivo: stimare il **prezzo** di un immobile a partire da alcune caratteristiche, tra cui la **superficie in m²**.

Nel notebook vedrai:
1. Un piccolo dataset di esempio (m², stanze, bagni, età)
2. Normalizzazione delle feature
3. Funzione di costo (MSE)
4. Calcolo dei gradienti
5. Gradient Descent passo-passo
6. Predizione su nuovi immobili e interpretazione dei pesi


## 1) Dataset di esempio
Ogni riga rappresenta un immobile.

**Feature (X):**
- `mq`: superficie in metri quadrati
- `stanze`: numero di stanze
- `bagni`: numero di bagni
- `eta`: età dell'immobile in anni

**Target (y):**
- `prezzo`: prezzo in euro (qui in *migliaia di euro* per mantenere numeri più piccoli)

> Nota: i dati sono fittizi ma realistici: servono per capire il metodo, non per fare stime di mercato.

In [1]:
# =========================================================
# 1) DATI
# =========================================================

# X_train: lista di esempi, ogni esempio è una lista di feature [mq, stanze, bagni, eta]
X_train = [
    [55,  2, 1, 35],
    [70,  3, 1, 20],
    [85,  3, 2, 10],
    [95,  4, 2,  8],
    [110, 4, 2, 15],
    [125, 5, 2,  5],
    [140, 5, 3,  3],
    [160, 6, 3, 12],
    [180, 6, 3,  6],
    [200, 7, 3,  2],
]

# y_train: prezzo in migliaia di euro (k€)
y_train = [150, 190, 260, 295, 310, 380, 430, 450, 520, 600]

m = len(X_train)      # numero di esempi
n = len(X_train[0])   # numero di feature

print(f"Esempi (m) = {m}, Feature (n) = {n}")
print("Prima riga X_train:", X_train[0], "-> y:", y_train[0], "k€")


Esempi (m) = 10, Feature (n) = 4
Prima riga X_train: [55, 2, 1, 35] -> y: 150 k€


## 2) Modello
Useremo il modello lineare multivariato:

y_hat(i) = w1 * x1(i) + w2 * x2(i) + ... + wn * xn(i) + b


dove:
x1, x2, ..., xn sono le feature dell'immobile
(esempio: metri quadrati, numero di stanze, numero di bagni, età)

w1, w2, ..., wn sono i pesi del modello
indicano quanto ogni feature influenza il prezzo

b è l'intercetta (bias)
serve a spostare la previsione verso l'alto o verso il basso

y_hat è il prezzo stimato dal modello

Implementiamo prima le funzioni di utilità: prodotto scalare, predizione e costo MSE.

In [2]:
# =========================================================
# 2) FUNZIONI DI BASE (no numpy)
# =========================================================
import math

def dot(a, b):
    """Prodotto scalare tra due liste di numeri della stessa lunghezza."""
    return sum(ai * bi for ai, bi in zip(a, b))

def predict_one(x, w, b):
    """Predizione per un singolo esempio x."""
    return dot(w, x) + b

def compute_cost_mse(X, y, w, b):
    """Costo MSE con fattore 1/(2m): J(w,b) = (1/(2m)) * sum((yhat - y)^2)."""
    m = len(X)
    total = 0.0
    for xi, yi in zip(X, y):
        yhat = predict_one(xi, w, b)
        err = yhat - yi
        total += err * err
    return total / (2 * m)

# inizializziamo w e b a zero
w = [0.0] * n
b = 0.0

J0 = compute_cost_mse(X_train, y_train, w, b)
print("Costo iniziale J(w=0,b=0) =", J0)


Costo iniziale J(w=0,b=0) = 73576.25


## 3) Perché normalizzare le feature

Le feature hanno scale molto diverse tra loro.
Ad esempio:
mq può andare da circa 50 a 200
stanze può andare da circa 2 a 7
eta può andare da circa 2 a 35
Quindi i valori hanno ordini di grandezza diversi.

Problema senza normalizzazione
Se lasciamo i valori così, il Gradient Descent può comportarsi male:
fa passi troppo grandi su alcune feature fa passi troppo piccoli su altre converge lentamente oppure può oscillare e non convergere bene
Questo rende l'apprendimento inefficiente.

Soluzione: normalizzazione con z-score

Trasformiamo ogni feature usando la formula:
x_norm = (x - mu) / sigma
dove:
x = valore originale della feature
mu = media della feature
sigma = deviazione standard della feature
x_norm = valore normalizzato

Cosa fa questa trasformazione

Dopo la normalizzazione:
la media diventa circa 0
la deviazione standard diventa circa 1
tutte le feature hanno scale simili

Questo rende il Gradient Descent molto più efficiente.

In [3]:
# =========================================================
# 3) NORMALIZZAZIONE (z-score) - solo standard library
# =========================================================
import statistics

def column_stats(X):
    """Ritorna (mu_list, sigma_list) per ciascuna colonna (feature)."""
    n = len(X[0])
    mus = []
    sigmas = []
    for j in range(n):
        col = [row[j] for row in X]
        mu = statistics.mean(col)
        # pstdev = deviazione standard "population" (divisione per m), stabile per ML didattico
        sigma = statistics.pstdev(col)
        # evitiamo divisione per zero nel caso sigma=0
        sigma = sigma if sigma != 0 else 1.0
        mus.append(mu)
        sigmas.append(sigma)
    return mus, sigmas

def normalize_features(X, mus, sigmas):
    """Applica z-score a tutto X."""
    Xn = []
    for row in X:
        Xn.append([(row[j] - mus[j]) / sigmas[j] for j in range(len(row))])
    return Xn

mus, sigmas = column_stats(X_train)
Xn_train = normalize_features(X_train, mus, sigmas)

print("Media (mu) per feature:", mus)
print("Dev std (sigma) per feature:", sigmas)
print("Esempio normalizzato (prima riga):")
print("  X originale:", X_train[0])
print("  X norm.:   ", [round(v, 3) for v in Xn_train[0]])


Media (mu) per feature: [122, 4.5, 2.2, 11.6]
Dev std (sigma) per feature: [45.45327270945405, 1.5, 0.7483314773547883, 9.414881836751857]
Esempio normalizzato (prima riga):
  X originale: [55, 2, 1, 35]
  X norm.:    [-1.474, -1.667, -1.604, 2.485]


## 4) Gradient Descent: idea e formule

Vogliamo trovare i valori migliori dei pesi w e dell'intercetta b che minimizzano la funzione di costo.
Ricorda che la funzione di costo è:
J(w,b) = (1/(2m)) * somma( (y_hat - y)^2 )

Per la regressione lineare, questa funzione ha una forma a "ciotola" (convessa), quindi esiste un solo minimo globale.
Il Gradient Descent è un algoritmo che trova questo minimo facendo piccoli passi nella direzione che riduce il costo.

4.1 Errore per ogni esempio
Per ogni immobile i:
errore(i) = y_hat(i) - y(i)
dove:
y_hat(i) = prezzo stimato
y(i) = prezzo reale

4.2 Gradiente rispetto ai pesi
Per ogni peso wj:
derivata_wj = (1/m) * somma( errore(i) * xj(i) )
dove:
m = numero di esempi
xj(i) = valore della feature j per l'esempio i
Questo valore indica quanto modificare il peso wj.

4.3 Gradiente rispetto all'intercetta
Per l'intercetta b:
derivata_b = (1/m) * somma( errore(i) )
Questo indica quanto modificare b.

4.4 Aggiornamento dei parametri
Aggiorniamo i parametri usando il tasso di apprendimento alpha:
wj = wj - alpha * derivata_wj
b = b - alpha * derivata_b

4.5 Significato intuitivo
Il Gradient Descent funziona così:
calcola l'errore
calcola in quale direzione muoversi
fa un piccolo passo in quella direzione
ripete il processo molte volte
Ad ogni iterazione il costo diminuisce.

4.6 Intuizione geometrica
Immagina di essere su una montagna:
il costo J è l'altezza
vuoi raggiungere il punto più basso
il gradiente indica la direzione di salita
andando nella direzione opposta, scendi verso il minimo

In [5]:
# =========================================================
# 4) GRADIENTI e GRADIENT DESCENT (no numpy)
# =========================================================

def compute_gradients(X, y, w, b):
    """Calcola dj/dw e dj/db (gradiente della funzione di costo)."""
    m = len(X)
    n = len(X[0])
    dj_dw = [0.0] * n
    dj_db = 0.0

    for xi, yi in zip(X, y):
        yhat = predict_one(xi, w, b)
        err = yhat - yi

        # accumula per b
        dj_db += err

        # accumula per ogni w_j
        for j in range(n):
            dj_dw[j] += err * xi[j]

    # media
    dj_db /= m
    dj_dw = [v / m for v in dj_dw]
    return dj_dw, dj_db

def gradient_descent(X, y, w_init, b_init, alpha, num_iters, print_every=100):
    """Esegue Gradient Descent e ritorna (w, b, history)."""
    w = w_init[:]
    b = b_init
    history = []

    for it in range(1, num_iters + 1):
        dj_dw, dj_db = compute_gradients(X, y, w, b)

        # aggiornamento simultaneo
        for j in range(len(w)):
            w[j] -= alpha * dj_dw[j]
        b -= alpha * dj_db

        # log del costo
        if it == 1 or it % print_every == 0 or it == num_iters:
            J = compute_cost_mse(X, y, w, b)
            history.append((it, J))
            print(f"Iter {it:4d} | J = {J:10.4f} | b = {b:8.4f} | w = {[round(v,4) for v in w]}")
    return w, b, history

# Iperparametri
alpha = 0.1
num_iters = 2000

# attenzione: usiamo X normalizzato!
w0 = [0.0] * n
b0 = 0.0

w_trained, b_trained, hist = gradient_descent(Xn_train, y_train, w0, b0, alpha, num_iters, print_every=200)


Iter    1 | J = 56135.0143 | b =  35.8500 | w = [13.5755, 13.3833, 12.6014, -10.6385]
Iter  200 | J =    85.6881 | b = 358.5000 | w = [72.9834, 43.0447, 14.5528, -8.6832]
Iter  400 | J =    68.1718 | b = 358.5000 | w = [86.9988, 32.0191, 9.6429, -10.7911]
Iter  600 | J =    57.6475 | b = 358.5000 | w = [97.6228, 22.6147, 7.1461, -12.2095]
Iter  800 | J =    51.1856 | b = 358.5000 | w = [105.9166, 15.1743, 5.3154, -13.2968]
Iter 1000 | J =    47.2167 | b = 358.5000 | w = [112.4136, 9.3367, 3.8923, -14.1466]
Iter 1200 | J =    44.7790 | b = 358.5000 | w = [117.505, 4.7611, 2.7782, -14.8124]
Iter 1400 | J =    43.2818 | b = 358.5000 | w = [121.4951, 1.1751, 1.9051, -15.3342]
Iter 1600 | J =    42.3622 | b = 358.5000 | w = [124.6222, -1.6353, 1.2209, -15.7431]
Iter 1800 | J =    41.7974 | b = 358.5000 | w = [127.073, -3.8378, 0.6846, -16.0636]
Iter 2000 | J =    41.4505 | b = 358.5000 | w = [128.9936, -5.564, 0.2644, -16.3147]


## 5) Interpretare il risultato
Abbiamo allenato il modello su feature **normalizzate**.

- Se un peso \(w_j\) è positivo, aumentando quella feature (in termini di deviazioni standard) il prezzo stimato aumenta.
- Se è negativo, l'effetto è opposto.

Attenzione però: con normalizzazione, i pesi non sono "€/m²" direttamente; sono "k€ per 1 deviazione standard" della feature.

Per fare predizioni su nuovi dati, dobbiamo:
1) prendere le feature grezze
2) normalizzarle con \(\mu\) e \(\sigma\) del training set
3) usare \(w\) e \(b\) allenati

In [10]:
# =========================================================
# 5) PREDIZIONE SU NUOVI IMMOBILI
# =========================================================

def normalize_one(x_raw, mus, sigmas):
    """Normalizza un singolo esempio x_raw usando mus e sigmas del training."""
    return [(x_raw[j] - mus[j]) / sigmas[j] for j in range(len(x_raw))]

def predict_price_k_euro(x_raw, w, b, mus, sigmas):
    """Ritorna la predizione in k€ per un immobile (feature grezze)."""
    x_norm = normalize_one(x_raw, mus, sigmas)
    return predict_one(x_norm, w, b)

# Esempio: immobile 100 m², 4 stanze, 2 bagni, 12 anni
x_new = [100, 4, 2, 12]
pred_k = predict_price_k_euro(x_new, w_trained, b_trained, mus, sigmas)

print("Nuovo immobile (raw):", x_new)
print("Prezzo stimato:", round(pred_k, 1), "k€  (~", round(pred_k*1000), "€ )")


Nuovo immobile (raw): [100, 4, 2, 12]
Prezzo stimato: 299.3 k€  (~ 299337 € )


## 6) Micro-demo numerica: 10 passi di discesa (per vedere il percorso)
Qui facciamo una mini-simulazione con **solo 10 iterazioni** e stampiamo:
- costo \(J\)
- valori di \(b\)
- primi due pesi \(w\) (giusto per tenere la stampa leggibile)

Questo aiuta gli studenti a vedere che il costo tende a scendere e che il minimo (per regressione lineare con MSE) è unico.

In [13]:
# =========================================================
# 6) MINI-DEMO: 10 step di gradient descent
# =========================================================

w_demo = [0.0] * n
b_demo = 0.0
alpha_demo = 0.1

print("Step |   J(w,b)   |    b    |   w[0] (mq)  |   w[1] (stanze) ")
for step in range(1, 11):
    dj_dw, dj_db = compute_gradients(Xn_train, y_train, w_demo, b_demo)
    for j in range(n):
        w_demo[j] -= alpha_demo * dj_dw[j]
    b_demo -= alpha_demo * dj_db

    J = compute_cost_mse(Xn_train, y_train, w_demo, b_demo)
    print(f"{step:>4d} | {J:>10.4f} | {b_demo:>7.3f} | {w_demo[0]:>12.3f} | {w_demo[1]:>13.3f}")


Step |   J(w,b)   |    b    |   w[0] (mq)  |   w[1] (stanze) 
   1 | 56135.0143 |  35.850 |       13.575 |        13.383
   2 | 44054.2856 |  68.115 |       22.550 |        22.158
   3 | 35122.1149 |  97.154 |       28.547 |        27.951
   4 | 28242.2406 | 123.288 |       32.615 |        31.815
   5 | 22816.9229 | 146.809 |       35.433 |        34.427
   6 | 18483.1189 | 167.978 |       37.438 |        36.228
   7 | 14997.3131 | 187.031 |       38.913 |        37.501
   8 | 12183.3523 | 204.178 |       40.042 |        38.429
   9 |  9907.3530 | 219.610 |       40.943 |        39.131
  10 |  8064.5404 | 233.499 |       41.692 |        39.683


## 7) Esercizio per studenti
1) Aggiungi o modifica 2–3 righe nel dataset (nuovi immobili) e ri-esegui l'allenamento.
2) Prova a cambiare \(\alpha\) (es. 0.01, 0.3) e osserva:
   - converge più lentamente?
   - diverge (costo che esplode)?
3) Prova a togliere la normalizzazione e confronta la stabilità.

Spunto di discussione:
- Quale feature sembra più influente sul prezzo? (guardando segno e grandezza dei pesi)
- Perché l'età potrebbe avere un peso negativo?